In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

     |████████████████████████████████| 911kB 5.4MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 

#importing libraries

In [ ]:
!pip install proxy-requests
!pip install selenium-wire

  Created wheel for proxy-requests: filename=proxy_requests-0.5.2-cp37-none-any.whl size=4199 sha256=bdd4863f2a2658fdb569f402e66f40bd9dfcb74b62877aa39f3c6e0d794c41a6
  Stored in directory: /root/.cache/pip/wheels/31/14/e3/efacb14f25301832d3c409e32727256d42d9d580bb476507e4
Successfully built proxy-requests
     |████████████████████████████████| 245kB 6.5MB/s 
     |████████████████████████████████| 112kB 46.5MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 3.2MB 45.0MB/s 
  Created wheel for kaitaistruct: filename=kaitaistruct-0.9-py2.py3-none-any.whl size=5513 sha256=2f4a00530f602ec9468208c2304b809f1eb7bf0b2c45cf7d38fec27939ebbc43
  Stored in directory: /root/.cache/pip/wheels/51/36/82/af4b5c1e88b575a90ca4472af36170b92424ff30e7ba8efbc2
  Created wheel for blinker: filename=blinker-1.4-cp37-none-any.whl size=13476 sha256=8f7d5a6a

In [ ]:
import requests
import json
import time
from urllib.parse import quote
from proxy_requests import ProxyRequests

In [ ]:
from seleniumwire import webdriver
chrome_options = webdriver.ChromeOptions()
#chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--incognito')

#setting up url and header

In [ ]:
def getting_headers(url,id):
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options) #sdownload and setup chromium driver path here 
  time.sleep(3)
  driver.get(url)
  time.sleep(20)
  data = driver.requests
  driver.quit()
  for i in data:
    if(('https://twitter.com/i/api/2/timeline/conversation/' + str(id) + '.json') in str(i)):
        index = data.index(i)
        header = dict(data[index].headers)
        del header['accept-encoding']
        return header

initial_url = 'https://twitter.com/i/api/2/timeline/conversation/'
initial_param = '.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweet=true&include_ext_has_birdwatch_notes=false&ext=mediaStats%2ChighlightedLabel'
count_param = '&count=20'
referrer_param = '&referrer=tweet'

def get_url(id,req_type):
  if(req_type==1):#initial
    url = initial_url + str(id) + initial_param + count_param
  elif(req_type==2 or req_type==4 or req_type==5):#thread replies or show replies vai button or show_button
    url =   initial_url + str(id) + initial_param + referrer_param
  elif(req_type==3):#bottom cursor
    url = initial_url + str(id) + initial_param + count_param + referrer_param
  return url


#get responce

In [ ]:
def get_responce(url, header):
  #print(header)
  
  data = requests.get(url,headers = header).json()
  
  #print(data)
  #data = json.load(data)
  #print(data)
  return data

#get comments and replies

In [ ]:
"""
1: initial
2: show more replies
3: bottom cursor
4: show replies cursor
"""
def get_comments(main_id,header,req_type,cursor = '',comm_level = 1,len_comm=0):
    print('# ',end='')
    comments = []
    if(cursor!=''):
        main_url = get_url(main_id,req_type)
        url = main_url + '&cursor=' + quote(cursor)
    else:
        url = get_url(main_id,req_type)
    data_main = get_responce(url, header)
    time.sleep(7)
    main_text = ''
    instruction_entries = data_main['timeline']['instructions'][0]['addEntries']['entries']
    if(cursor=='' and comm_level==1):
        main_text = data_main['globalObjects']['tweets'][main_id]['full_text']
        comments.append('main_tweet:'+main_text)
    #print(comments)
    #print()
    #print()
    for elem in instruction_entries:
        #if(len_comm>10 and comm_level==1):
        #  return comments
        thread = []
        if('timelineModule' in elem['content']):
            for item in elem['content']['timelineModule']['items']:
                if('tweet' in item['entryId']):
                    if('tweet' in item['item']['content']):
                        id = item['item']['content']['tweet']['id']
                        thread.append({'tweet_id':id,'tweet':data_main['globalObjects']['tweets'][id]['full_text']})
                        if(data_main['globalObjects']['tweets'][id]['reply_count']>0 and comm_level==1):
                          c = get_comments(id,header,1,cursor = '',comm_level = 2,len_comm=0)
                          thread[0]['replies'] = c
                if(len(thread)==1):
                  break
            comments.append(thread)
        len_comm+=1
        if('cursor-bottom' in elem['entryId']):
            cursor = elem['content']['operation']['cursor']['value']
            comm = get_comments(main_id,header,3, cursor,comm_level,len_comm)
            comments = comments + comm
                
        if('cursor-showMoreThreads' in elem['entryId']):
            cursor = elem['content']['operation']['cursor']['value']
            comm = get_comments(main_id,header,4, cursor,comm_level,len_comm)
            comments = comments + comm
                
        if('cursor-showMoreThreadsPrompt' in elem['entryId']):
            cursor = elem['content']['operation']['cursor']['value']
            comm = get_comments(main_id,header,5, cursor,comm_level,len_comm)
            comments = comments + comm
        
    return comments  


#scraping data

In [ ]:
def get_data(url,savepath = '',exception_count = 0):
    id = url.split('/')[-1]
    temp_data = {}
    temp = {}
    try:
        header = getting_headers(url,id)
        comm = get_comments(id,header,1)
        temp_data['comments'] = comm
#        print(temp_data)
        print('data gatharing done')
        with open('demo.json', 'w') as outfile:
          json.dump(temp_data,outfile)
        main_tweet = temp_data['comments'][0].strip('main_tweet:')
        temp['tweet'] = main_tweet
        temp['tweet_id'] = id
        del temp_data['comments'][0]
        temp['comments'] = []
        for comments in temp_data['comments']:
          if(len(comments)>0):
            comment_tweet = comments[0]['tweet']
            comment_tweet_id = comments[0]['tweet_id']
            if('replies' in comments[0]):
              rep = []
              for r in comments[0]['replies']:
                if(len(r)>0):
                  rep.append({'tweet_id':r[0]['tweet_id'],'tweet':r[0]['tweet']})
              #rep = set(rep)
              #rep = list(rep)
              temp['comments'].append({'tweet_id':comment_tweet_id, 'tweet':comment_tweet,'replies':rep})
            else:
              temp['comments'].append({'tweet_id':comment_tweet_id, 'tweet':comment_tweet})
        #cmm = {}
        #cmm['com'] = comm
        print(str(id) + ' : success : ', end ='')
        with open(savepath + str(id) + '.json','w',encoding='utf8') as outfile: #modify filepath and filename here
            json.dump(temp, outfile,ensure_ascii=False)
            print('saved')
    except:
        print(' = = = = = = = = = = = = = Exception = = = = = = = = = = = = =')
        time.sleep(100)
        exception_count+=1
        if(exception_count==3):
          return
        get_data(url,savepath,exception_count)


In [ ]:
get_data('https://twitter.com/ReallySwara/status/1413974963535552512', savepath = '/content/drive/MyDrive/hasoc2021/tweets_with_limited_comments/')

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # data gatharing done
1413974963535552512 : success : saved


In [ ]:
url_list = []
with open('/content/drive/MyDrive/hasoc2021/baps.txt', 'r') as filehandle:
    for line in filehandle:
        if(line!=''):
          currentPlace = line[:-1]
          url_list.append(currentPlace)
print(url_list)

for tweet_url in url_list:
    get_data(tweet_url, savepath = '/content/drive/MyDrive/hasoc2021/tweets_with_limited_comments/baps/')

['https://twitter.com/Profdilipmandal/status/1392725164060594178', 'https://twitter.com/RadhaTr98/status/139353568926273126']
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # data gatharing done
1392725164060594178 : success : saved
#  = = = = = = = = = = = = = Exception = = = = = = = = = = = = =


KeyboardInterrupt: ignored

In [ ]:
url_list = []
with open('/content/drive/MyDrive/hasoc2021/godse.txt', 'r') as filehandle:
    for line in filehandle:
        if(line!=''):
          currentPlace = line[:-1]
          url_list.append(currentPlace)
print(url_list)

for tweet_url in url_list:
    get_data(tweet_url, savepath = '/content/drive/MyDrive/hasoc2021/tweets_with_limited_comments/godse/')

['https://twitter.com/AmitsharmaGRENO/status/1394942092325974020', 'https://twitter.com/TheDeepak2021ln/status/1394915940077162496', 'https://twitter.com/Mr_madhavendra/status/1394930678106492931', 'https://twitter.com/SrivastavaS08/status/139491540012763136']
# # # # # # # # # # # # # # # # # # data gatharing done
1394942092325974020 : success : saved
# # # # # # # # data gatharing done
1394915940077162496 : success : saved
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # data gatharing done
1394930678106492931 : success : saved
#  = = = = = = = = = = = = = Exception = = = = = = = = = = = = =
#  = = = = = = = = = = = = = Exception = = = = = = = = = = = = =
#  = = = = = = = = = = = = = Exception = = = = = = = = = = = = =


In [ ]:
url_list = []
with open('/content/drive/MyDrive/hasoc2021/kejri_modi.txt', 'r') as filehandle:
    for line in filehandle:
        if(line!=''):
          currentPlace = line[:-1]
          url_list.append(currentPlace)
print(url_list)

for tweet_url in url_list:
    get_data(tweet_url, savepath = '/content/drive/MyDrive/hasoc2021/tweets_with_limited_comments/kejri_modi/')

['https://twitter.com/ArvindKejriwal/status/676627652204060672', 'https://twitter.com/alimehdi_inc/status/1390979182608326658', 'https://twitter.com/HemantSorenJMM/status/1390362987995623428', 'https://twitter.com/desh_bhkt/status/139493183097805619']
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # data gatharing done
676627652204060672 : success : saved
# # # data gatharing done
1390979182608326658 : success : saved
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

In [ ]:
url_list = []
with open('/content/drive/MyDrive/hasoc2021/asharam.txt', 'r') as filehandle:
    for line in filehandle:
        if(line!=''):
          currentPlace = line[:-1]
          url_list.append(currentPlace)
print(url_list)

for tweet_url in url_list:
    get_data(tweet_url, savepath = '/content/drive/MyDrive/hasoc2021/tweets_with_limited_comments/asharam/')

['https://twitter.com/BrijPal43219651/status/1393764797859078144', 'https://twitter.com/ChelaniSakshi/status/1393528907102515200', 'https://twitter.com/azizkavish/status/1391768816355446787', 'https://twitter.com/_mohammad_ali_7/status/1393625939741794306', 'https://twitter.com/pihu0297/status/139123495624071987']
# # # # # # # data gatharing done
1393764797859078144 : success : saved
# # # # # # # # # # # # # data gatharing done
1393528907102515200 : success : saved
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # data gatharing done
1391768816355446787 : success : saved
# # # data gatharing done
1393625939741794306 : success : saved
#  = = = = = = = = = = = = = Exception = = = = = = = = = = = = =
#  = = = = = = = = = = = = = Exception = = = = = = = = = = = = =
#  = = = = = = = = = = = = = Exception = = = = = = = = = = = = =


In [ ]:
url_list = []
with open('/content/drive/MyDrive/hasoc2021/modi_badword.txt', 'r') as filehandle:
    for line in filehandle:
        if(line!=''):
          currentPlace = line[:-1]
          url_list.append(currentPlace)
print(url_list)

for tweet_url in url_list:
    get_data(tweet_url, savepath = '/content/drive/MyDrive/hasoc2021/tweets_with_limited_comments/modi_badword/')

['https://twitter.com/jkhothi/status/1394390667514712065', 'https://twitter.com/Akki_SwagMan/status/1393620880840921088', 'https://twitter.com/RoopsSaxena/status/1395677189870211073', 'https://twitter.com/AcharyaPramodk/status/139594128363436851']
# # # # data gatharing done
1394390667514712065 : success : saved
# # data gatharing done
1393620880840921088 : success : saved
# # # # # # # # # # data gatharing done
1395677189870211073 : success : saved
#  = = = = = = = = = = = = = Exception = = = = = = = = = = = = =
#  = = = = = = = = = = = = = Exception = = = = = = = = = = = = =
#  = = = = = = = = = = = = = Exception = = = = = = = = = = = = =


In [ ]:
url_list = []
with open('/content/drive/MyDrive/hasoc2021/covid-crisis.txt', 'r') as filehandle:
    for line in filehandle:
        if(line!=''):
          currentPlace = line[:-1]
          url_list.append(currentPlace)
print(url_list)

for tweet_url in url_list:
    get_data(tweet_url, savepath = '/content/drive/MyDrive/hasoc2021/tweets_with_limited_comments/covid-crisis/')

['https://twitter.com/sudhirchaudhary/status/1395277604782022659', 'https://twitter.com/MahuaMoitra/status/1386182358219460609', 'https://twitter.com/MahuaMoitra/status/1385779864649691136', 'https://twitter.com/MahuaMoitra/status/1385527614857646083', 'https://twitter.com/MahuaMoitra/status/1383261325531353098', 'https://twitter.com/narendramodi/status/1385891002209038343', 'https://twitter.com/sambitswaraj/status/1394528104492658700', 'https://twitter.com/POTUS/status/1386401947729633280', 'https://twitter.com/HansrajMeena/status/1391990111223500802', 'https://twitter.com/SharjeelUsmani/status/1391494336534093826', '', '']
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # data gatharing done
13952

<h1>DEBUG

In [ ]:
#modify structure
import json
f = open('potus_updated.json')
data = json.load(f)
temp = {}
main_tweet = data['comments'][0].strip('main_tweet:')
temp['title'] = main_tweet
del data['comments'][0]

"""temp['comments'] = []
for comments in data['comments']:
    temp2 = {}
    if(len(comments)>0):
        temp2['comment'] = comments[0]
        del comments[0]
        if(len(comments)>0):
            temp2['replies'] = comments
        temp['comments'].append(temp2)"""
with open('./hasoc_tweets/1385595796662460416.json','w') as openfile:
    json.dump(temp,openfile)

In [ ]:
get_data('https://twitter.com/adarpoonawalla/status/1382978713302683653')

# {'globalObjects': {'tweets': {'1382982119576637444': {'created_at': 'Fri Apr 16 09:00:06 +0000 2021', 'id_str': '1382982119576637444', 'full_text': '@JyotiTathgur @adarpoonawalla @POTUS You mean requesting via Twitter instead of formal diplomatic channels?\n\nAdvisory and consulting M0R0NS 😏\n\nhttps://t.co/RiuWSOMbK6', 'display_text_range': [37, 166], 'entities': {'user_mentions': [{'screen_name': 'JyotiTathgur', 'name': 'Jyoti Tathgur', 'id_str': '967402298174574592', 'indices': [0, 13]}, {'screen_name': 'adarpoonawalla', 'name': 'Adar Poonawalla', 'id_str': '911182427782909952', 'indices': [14, 29]}, {'screen_name': 'POTUS', 'name': 'President Biden', 'id_str': '1349149096909668363', 'indices': [30, 36]}], 'urls': [{'url': 'https://t.co/RiuWSOMbK6', 'expanded_url': 'https://m.economictimes.com/industry/healthcare/biotech/healthcare/india-takes-up-with-us-ban-on-exports-of-vax-raw-materials/articleshow/81473965.cms', 'display_url': 'm.economictimes.com/industry/healt…', 'indices': 

# {'globalObjects': {'tweets': {'1384302852009385993': {'created_at': 'Tue Apr 20 00:28:13 +0000 2021', 'id_str': '1384302852009385993', 'full_text': '@Mandar_240Bhp @adarpoonawalla @POTUS https://t.co/nF4KGT0cPO', 'display_text_range': [37, 37], 'entities': {'user_mentions': [{'screen_name': 'Mandar_240Bhp', 'name': 'Mandar', 'id_str': '429973227', 'indices': [0, 14]}, {'screen_name': 'adarpoonawalla', 'name': 'Adar Poonawalla', 'id_str': '911182427782909952', 'indices': [15, 30]}, {'screen_name': 'POTUS', 'name': 'President Biden', 'id_str': '1349149096909668363', 'indices': [31, 37]}], 'media': [{'id_str': '1384302842035400712', 'indices': [38, 61], 'media_url': 'http://pbs.twimg.com/tweet_video_thumb/EzYIGbkVcAgauLi.jpg', 'media_url_https': 'https://pbs.twimg.com/tweet_video_thumb/EzYIGbkVcAgauLi.jpg', 'url': 'https://t.co/nF4KGT0cPO', 'display_url': 'pic.twitter.com/nF4KGT0cPO', 'expanded_url': 'https://twitter.com/syedfaisal1333/status/1384302852009385993/photo/1', 'type': 'photo

# {'globalObjects': {'tweets': {'1383104182664654848': {'created_at': 'Fri Apr 16 17:05:08 +0000 2021', 'id_str': '1383104182664654848', 'full_text': '@babuBORA78 @Pradeep76564779 @adarpoonawalla @POTUS @PMOIndia @narendramodi Apart from Modi ji? Ig it was Russia', 'display_text_range': [76, 112], 'entities': {'user_mentions': [{'screen_name': 'babuBORA78', 'name': 'Biswajit Bora', 'id_str': '807598405740613632', 'indices': [0, 11]}, {'screen_name': 'Pradeep76564779', 'name': 'Pradeep Kumar K', 'id_str': '1179987931018096641', 'indices': [12, 28]}, {'screen_name': 'adarpoonawalla', 'name': 'Adar Poonawalla', 'id_str': '911182427782909952', 'indices': [29, 44]}, {'screen_name': 'POTUS', 'name': 'President Biden', 'id_str': '1349149096909668363', 'indices': [45, 51]}, {'screen_name': 'PMOIndia', 'name': 'PMO India', 'id_str': '471741741', 'indices': [52, 61]}, {'screen_name': 'narendramodi', 'name': 'Narendra Modi', 'id_str': '18839785', 'indices': [62, 75]}]}, 'source': '<a href="http:/

# {'globalObjects': {'tweets': {'1383018949130870786': {'created_at': 'Fri Apr 16 11:26:27 +0000 2021', 'id_str': '1383018949130870786', 'full_text': '@NasharatM @ahamasmi0 @4BH1786 @Tweeter_oo @Wentalist @prasadhemant @JyotiTathgur @adarpoonawalla @POTUS https://t.co/nJmBCI9BGd', 'display_text_range': [104, 104], 'entities': {'user_mentions': [{'screen_name': 'NasharatM', 'name': '#Andolanjivi Nash', 'id_str': '1291048022881656833', 'indices': [0, 10]}, {'screen_name': 'ahamasmi0', 'name': 'Man dar', 'id_str': '2910302532', 'indices': [11, 21]}, {'screen_name': '4BH1786', 'name': 'INTOLERANT LIBERAL', 'id_str': '1325343318465679366', 'indices': [22, 30]}, {'screen_name': 'Tweeter_oo', 'name': 'KazuriKazuri', 'id_str': '1043476958271528960', 'indices': [31, 42]}, {'screen_name': 'Wentalist', 'name': 'Coins and Cards', 'id_str': '1377116456291168262', 'indices': [43, 53]}, {'screen_name': 'prasadhemant', 'name': 'Hemant Prasad', 'id_str': '114707915', 'indices': [54, 67]}, {'screen_name

# {'globalObjects': {'tweets': {'1383106063344406529': {'created_at': 'Fri Apr 16 17:12:37 +0000 2021', 'id_str': '1383106063344406529', 'full_text': "@aimingforlight @akki_rane @adarpoonawalla @POTUS @DrSJaishankar prolly was too young in march'21 to read newspapers or surf the net. 😜", 'display_text_range': [65, 135], 'entities': {'user_mentions': [{'screen_name': 'aimingforlight', 'name': 'Agnostic_Exploring', 'id_str': '767860748', 'indices': [0, 15]}, {'screen_name': 'akki_rane', 'name': 'Akshay Rane', 'id_str': '142969190', 'indices': [16, 26]}, {'screen_name': 'adarpoonawalla', 'name': 'Adar Poonawalla', 'id_str': '911182427782909952', 'indices': [27, 42]}, {'screen_name': 'POTUS', 'name': 'President Biden', 'id_str': '1349149096909668363', 'indices': [43, 49]}, {'screen_name': 'DrSJaishankar', 'name': 'Dr. S. Jaishankar', 'id_str': '859363899291742210', 'indices': [50, 64]}]}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_

...

# {'globalObjects': {'tweets': {'1382978713302683653': {'created_at': 'Fri Apr 16 08:46:34 +0000 2021', 'id_str': '1382978713302683653', 'full_text': 'Respected @POTUS, if we are to truly unite in beating this virus, on behalf of the vaccine industry outside the U.S., I humbly request you to lift the embargo of raw material exports out of the U.S. so that vaccine production can ramp up. Your administration has the details. 🙏🙏', 'display_text_range': [0, 278], 'entities': {'user_mentions': [{'screen_name': 'POTUS', 'name': 'President Biden', 'id_str': '1349149096909668363', 'indices': [10, 16]}]}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'user_id_str': '911182427782909952', 'retweet_count': 27741, 'favorite_count': 107785, 'reply_count': 2332, 'quote_count': 2492, 'conversation_id_str': '1382978713302683653', 'lang': 'en'}, '1383059974004248582': {'created_at': 'Fri Apr 16 14:09:28 +0000 2021', 'id_str': '1383059974004248582', 'full_text': '@a

# {'globalObjects': {'tweets': {'1382978713302683653': {'created_at': 'Fri Apr 16 08:46:34 +0000 2021', 'id_str': '1382978713302683653', 'full_text': 'Respected @POTUS, if we are to truly unite in beating this virus, on behalf of the vaccine industry outside the U.S., I humbly request you to lift the embargo of raw material exports out of the U.S. so that vaccine production can ramp up. Your administration has the details. 🙏🙏', 'display_text_range': [0, 278], 'entities': {'user_mentions': [{'screen_name': 'POTUS', 'name': 'President Biden', 'id_str': '1349149096909668363', 'indices': [10, 16]}]}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'user_id_str': '911182427782909952', 'retweet_count': 27741, 'favorite_count': 107785, 'reply_count': 2332, 'quote_count': 2492, 'conversation_id_str': '1382978713302683653', 'lang': 'en'}, '1383062570026749959': {'created_at': 'Fri Apr 16 14:19:47 +0000 2021', 'id_str': '1383062570026749959', 'full_text': '@a

# {'globalObjects': {'tweets': {'1383294984179224580': {'created_at': 'Sat Apr 17 05:43:19 +0000 2021', 'id_str': '1383294984179224580', 'full_text': '@bold_views @adarpoonawalla @POTUS The ones India is exporting outside like #vaccineMaitri is what GOI paid ones not pvt enterprise. Foreign funded ones are what Serum exported to Pakistan. Clear distinction between all. Adar complaint abt raw materials is completely new Vaccine.', 'display_text_range': [35, 280], 'entities': {'hashtags': [{'text': 'vaccineMaitri', 'indices': [76, 90]}], 'user_mentions': [{'screen_name': 'bold_views', 'name': 'Bold Views', 'id_str': '3301435379', 'indices': [0, 11]}, {'screen_name': 'adarpoonawalla', 'name': 'Adar Poonawalla', 'id_str': '911182427782909952', 'indices': [12, 27]}, {'screen_name': 'POTUS', 'name': 'President Biden', 'id_str': '1349149096909668363', 'indices': [28, 34]}]}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_i

# {'globalObjects': {'tweets': {'1383065810604761088': {'created_at': 'Fri Apr 16 14:32:40 +0000 2021', 'id_str': '1383065810604761088', 'full_text': '@adarpoonawalla We understand that SII is one of the best manufacturers of vaccines and biological but the protocol needs to be maintained. @drharshvardhan and @PMOIndia may kindly take up this issue with concerned country.', 'display_text_range': [16, 223], 'entities': {'user_mentions': [{'screen_name': 'adarpoonawalla', 'name': 'Adar Poonawalla', 'id_str': '911182427782909952', 'indices': [0, 15]}, {'screen_name': 'drharshvardhan', 'name': 'Dr Harsh Vardhan', 'id_str': '77732185', 'indices': [140, 155]}, {'screen_name': 'PMOIndia', 'name': 'PMO India', 'id_str': '471741741', 'indices': [160, 169]}]}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id_str': '1383063770495209473', 'in_reply_to_user_id_str': '934888289659641856', 'in_reply_to_screen_name': 'nirajeva', 'us

# {'globalObjects': {'tweets': {'1383069923576926219': {'created_at': 'Fri Apr 16 14:49:00 +0000 2021', 'id_str': '1383069923576926219', 'full_text': '@sricon @adarpoonawalla @POTUS @narendramodi @PMOIndia @adarpoonawalla asking Potus. Why?', 'display_text_range': [55, 89], 'entities': {'user_mentions': [{'screen_name': 'sricon', 'name': 'Srikanth P', 'id_str': '53293993', 'indices': [0, 7]}, {'screen_name': 'adarpoonawalla', 'name': 'Adar Poonawalla', 'id_str': '911182427782909952', 'indices': [8, 23]}, {'screen_name': 'POTUS', 'name': 'President Biden', 'id_str': '1349149096909668363', 'indices': [24, 30]}, {'screen_name': 'narendramodi', 'name': 'Narendra Modi', 'id_str': '18839785', 'indices': [31, 44]}, {'screen_name': 'PMOIndia', 'name': 'PMO India', 'id_str': '471741741', 'indices': [45, 54]}, {'screen_name': 'adarpoonawalla', 'name': 'Adar Poonawalla', 'id_str': '911182427782909952', 'indices': [55, 70]}]}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">

In [ ]:
main_id = "1397578941041311749"
header = getting_headers("https://twitter.com/rizwankhan17_/status/1397578941041311749",main_id)

c = get_comments(main_id,header,1,cursor = '',comm_level = 2,len_comm=0)

# # 

In [ ]:
c

[[{'tweet': '@rizwankhan17_ Okay! Rizwan bhai..at least u can say thx to France🇫🇷 for helping us\noh sorry!! 😁',
   'tweet_id': '1397618474046787585'}],
 [{'tweet': '@rizwankhan17_ @narendramodi @EmmanuelMacron Centre gave but unfortunately vaccines r being wasted by chhattisgarh, jharkhand nd tn ( top states)',
   'tweet_id': '1397579242402062338'}],
 [{'tweet': '@rizwankhan17_ @narendramodi @EmmanuelMacron In Jharkhand and chatthisgarh about 37% &amp; 30% of the vaccines have been wasted',
   'tweet_id': '1397579451022528515'}],
 [{'tweet': '@rizwankhan17_ @narendramodi @EmmanuelMacron Achha naam hai... Punctureputra 😅😅😅',
   'tweet_id': '1397579835782828041'}],
 [{'tweet': '@rizwankhan17_ @narendramodi @EmmanuelMacron State khud ni kharid skte unko kya ande dene k lie vote diya h?',
   'tweet_id': '1397580555160461316'}],
 [{'tweet': '@rizwankhan17_ @narendramodi @EmmanuelMacron Pelestiniyo Ko yaha vaccine nhi milti h',
   'tweet_id': '1397580809914130435'}],
 [{'tweet': '@rizwankha